In [1]:
import os
import numpy as np
import scanpy as sc
import anndata as ad
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import warnings
plt.rcParams['figure.figsize']=(5, 5)
plt.rcParams['figure.dpi'] = 100
sc.settings.verbosity = 3
sc.logging.print_versions()
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2 

scanpy==1.4.4.post1 anndata==0.7.4 umap==0.3.10 numpy==1.19.2 scipy==1.5.2 pandas==1.1.2 scikit-learn==0.23.2 statsmodels==0.12.0 python-igraph==0.8.2 louvain==0.6.1


In [2]:
import sys
sys.path.append('./../../code/') 

In [3]:
from mast_scripts import *

In [4]:
import rpy2.rinterface_lib.callbacks
import logging
import warnings
from rpy2.robjects import pandas2ri
import anndata2ri

# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [5]:
%%R
# Load all the R libraries we will be using in the notebook
library(MAST)
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 10 (buster)

Matrix products: default
BLAS:   /app/R36/lib/R/lib/libRblas.so
LAPACK: /app/R36/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
 [1] parallel  stats4    tools     stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] MAST_1.12.0                 SingleCellExperiment_1.8.0 
 [3] SummarizedExperiment_1.16.1 DelayedArray_0.12.3        
 [5] BiocParallel_1.20.1         matrixStats_0.56.0         
 [7] Biobase_2.46.0              GenomicRanges_1.38.0       
 [9] GenomeInf

In [ ]:
adata = sc.read('./../../data/processed/pvn_renamed_latest.h5ad')

In [22]:
de_bulk = mast_de_bulk(adata, save='./de_pvn_bulk.xlsx')

[1] "Deploying to R..."
[1] "Begin computation..."


In [ ]:
de = mast_de_groups(adata, groupby='louvain_r1', save='./de_pvn_.xlsx')

[1] "Deploying to R..."
[1] "Begin computation..."
[1] "Fasciculata 1"
[1] "Fasciculata 2"
[1] "Fasciculata 3"
[1] "Fasciculata 4"
[1] "Fasciculata 5"
[1] "Glomerulosa 1"
[1] "Transition 1"
